In [1]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd
from scipy.misc import imread, toimage
from sklearn.metrics import accuracy_score
import gzip
from mnist import MNIST
from tk_package.utils.helper import dense_to_one_hot, preproc, batch_creator, show

root_dir = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
data_dir = os.path.join(root_dir, 'data')
sub_dir = os.path.join(root_dir, 'sub')

seed = 128
rng = np.random.RandomState(seed)

mndata = MNIST(data_dir)

In [3]:
X_train, y_train = mndata.load_training()
X_test, y_test = mndata.load_testing()

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

split_size = int(len(X_train)*0.7)
train_x, val_x = X_train[:split_size,:], X_train[split_size:,:]
train_y, val_y = y_train[:split_size], y_train[split_size:]

In [10]:
def cnn_model_fn(features, labels, mode):
    
    # define input layer
    input_layer = tf.reshape(features['x'], [-1, 28, 28, 1])
    
    # define the first convolutional layer
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding='same',
        activation=tf.nn.relu
    )
    
    # define the first pooling layer
    pool1 = tf.layers.max_pooling2d(
        inputs=conv1,
        pool_size=[2, 2],
        strides=2
    )
    
    # define the second convolutional layer
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding='same',
        activation=tf.nn.relu
    )
    
    # define the second pooling layer
    pool2 = tf.layers.max_pooling2d(
        inputs=conv2,
        pool_size=[2, 2],
        strides=2
    )
    
    # define the dense layer
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.dense(
        inputs=pool2_flat,
        units=1024,
        activation=tf.nn.relu
    )
    dropout = tf.layers.dropout(
        inputs=dense,
        rate=0.4,
        training=mode == tf.estimator.ModeKeys.TRAIN
    )
    
    # define the logits layer
    logits = tf.layers.dense(
        inputs=dropout, 
        units=10
    )
    
    # define the predictions
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name='softmax_tensor')
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    # define one hot encoding
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    
    # calculate loss (for both TRAIN and EVAL)
    loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)
    
    # configure the training op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step()
        )
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    # add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels=labels, predictions=predictions['classes']
        )
    }
    return tf.estimator.EstimatorSpec(
        mode=mode, 
        loss=loss,
        eval_metric_ops=eval_metric_ops
    )

def main():
    # Load training and eval data
    train_data = np.asarray(train_x, dtype=np.float32)
    train_labels = train_y
    eval_data = val_x
    eval_labels = val_y
    
#     tensors_to_log = {"probabilities": "softmax_tensor"}
#     logging_hook = tf.train.LoggingTensorHook(
#       tensors=tensors_to_log, every_n_iter=50)
    
    mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": train_data},
        y=train_labels,
        batch_size=100,
        num_epochs=None,
        shuffle=True)
    mnist_classifier.train(
        input_fn=train_input_fn,
        steps=20000)
#         hooks=[logging_hook])
    
    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": np.asarray(eval_data, dtype=np.float32)},
        y=eval_labels,
        num_epochs=1,
        shuffle=False)
    eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)

main()

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd94c77bfd0>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-60000
INFO:tensorflow:Saving checkpoints for 60001 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:loss = 0.00154454, step = 60001
INFO:tensorflow:global_step/sec: 188.856
INFO:tensorflow:loss = 0.00245036, step = 60101 (0.531 sec)
INFO:tensorflow:global_step/sec: 212.66
INFO:tens